In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import det, inv, svd, norm
import random

In [ ]:
colors = ['(255, 140, 0)', '(255, 69, 0)', '(154, 205, 50)', '(128, 128, 0)', '(85, 107, 47)',
              '(0, 128, 128)', '(70, 130, 180)', '(139, 69, 19)', '(105, 105, 105)']
random.shuffle(colors)
colors = [tuple(map(lambda x: int(x) / 255, color[1:-1].split(', '))) for color in colors]

glob_vals = []

In [ ]:
# parameter continuation method

def lambda_iterate(linsp, D, K, A, init_approx, system, bc, draw_nth, filename):
    global k11, k12, k13, k21, k22, k23
    k11, k12, k13, k21, k22, k23 = K
    SignChangeFile = open(addpath + '/ProjFiles/SignChange.txt', 'a')
    Lambdas0File = open(addpath + '/ProjFiles/Lambdas0File_{D}_{k11}_{k12}_{k13}.txt'.format(D = D, k11 = k11, k12 = k12, k13 = k13), 'a')
    print('\n', 'linspace ({l1}, {l2}, {l3})------'.format(l1 = linsp[0], l2 = linsp[1], l3 = linsp[2]), '\n', file = Lambdas0File)
    
    global la
    params, bc_params = [], []
    lambdas = np.linspace(linsp[0], linsp[1], linsp[2])
    lambdas_for_plot, dets = [], []
    lambdas_mistake_zerodet, lambdas_mistake_nan, lambdas_mistake_cycle = [], [], []
    abcFile = open(filename, 'a')
    y_approx = init_approx
    cnt = 0
    for la in lambdas:
        
        cnt += 1
        print(la, file = abcFile)
        result = shooting(np.linspace(-A, A, 100), y_approx, system, params, bc, bc_params, filename)
        if result[0] != 0:
            print('la = ', la, 'mistake', file = abcFile)
            if result[0] == 1:
                # -----
                # grkutta(system, result[1], np.linspace(-A, A, 100), 'la = {lamb}'.format(lamb = round(la, 2)))    # ---------------------
                # -----
                lambdas_mistake_zerodet.append(la) # det ~= 0
            if result[0] == 2:
                lambdas_mistake_nan.append(la) # nans in matrix
            if result[0] == 3:
                lambdas_mistake_cycle.append(la) # too many steps
            continue
        if cnt % draw_nth == 0:
            global glob_vals
            print("y_approx = ", y_approx, " la = ", la)  # ------------------------------  print y_approx
            steps, yval, zval = grkutta(system, result[1], np.linspace(-A, A, 100), 'la = {lamb}'.format(lamb = round(la, 2)))
            glob_vals.append([steps, yval, zval])
            
        # saving changing of determinant sign
        if len(dets) > 0 and result[3] * dets[-1] < 0:
            print('\n', D, K, A, y_approx, la, file = SignChangeFile)
            print('Det Change, lambda = {lamb}'.format(lamb = round(la, 2)))
            print("y_approx = ", y_approx, " la = ", la)  # ------------------------------  print y_approx
            steps, yval, zval = grkutta(system, result[1], np.linspace(-A, A, 100), '')
            #---
            glob_vals.append([steps, yval, zval])
            #----
            print(list(steps), '\n', yval, '\n', zval, '\n', file = SignChangeFile)
            print(la, '\n', file = Lambdas0File)
        y_approx = result[1]
        print("det = ", result[3], file = abcFile)
        lambdas_for_plot.append(la)
        dets.append(result[3])
    abcFile.close()
    SignChangeFile.close()
    Lambdas0File.close()
    return dets, lambdas_for_plot, lambdas_mistake_zerodet, lambdas_mistake_nan, lambdas_mistake_cycle

def lambda_iterate_svd(linsp, D, K, A, init_approx, system, bc, draw_nth, filename):
    global k11, k12, k13, k21, k22, k23
    k11, k12, k13, k21, k22, k23 = K
    SignChangeFile = open(addpath + '/ProjFiles/SignChange.txt', 'a')
    Lambdas0File = open(addpath + '/ProjFiles/Lambdas0File_{D}_{k11}_{k12}_{k13}.txt'.format(D = D, k11 = k11, k12 = k12, k13 = k13), 'a')
    print('\n', 'linspace ({l1}, {l2}, {l3})------'.format(l1 = linsp[0], l2 = linsp[1], l3 = linsp[2]), '\n', file = Lambdas0File)
    
    global la
    params, bc_params = [], []
    lambdas = np.linspace(linsp[0], linsp[1], linsp[2])
    lambdas_for_plot, dets = [], []
    first_sinval = []
    num_sing_values = []
    lam_steps = []
    lambdas_mistake_zerodet, lambdas_mistake_nan, lambdas_mistake_cycle = [], [], []
    abcFile = open(filename, 'a')
    y_approx = init_approx
    cnt = 0
    for la in lambdas:
        
        cnt += 1
        print(la, file = abcFile)
        result = shooting(np.linspace(-A, A, 100), y_approx, system, params, bc, bc_params, filename)
        if result[0] != 0:
            print('la = ', la, 'mistake', file = abcFile)
            if result[0] == 1:
                # -----
                # grkutta(system, result[1], np.linspace(-A, A, 100), 'la = {lamb}'.format(lamb = round(la, 2)))    # ---------------------
                # -----
                lambdas_mistake_zerodet.append(la) # det ~= 0
                lam_steps.append(la)
                first_sinval.append(result[-1][0])
                num_sing_values.append(len(result[-1]))
            if result[0] == 2:
                lambdas_mistake_nan.append(la) # nans in matrix
            if result[0] == 3:
                lambdas_mistake_cycle.append(la) # too many steps
            continue
        lam_steps.append(la)
        first_sinval.append(result[-1][0])
        num_sing_values.append(len(result[-1]))
        if cnt % draw_nth == 0:
            global glob_vals
            print("y_approx = ", y_approx, " la = ", la)  # ------------------------------  print y_approx
            steps, yval, zval = grkutta(system, result[1], np.linspace(-A, A, 100), 'la = {lamb}'.format(lamb = round(la, 2)))
            glob_vals.append([steps, yval, zval])
            
        # saving changing of determinant sign
        if len(dets) > 0 and result[3] * dets[-1] < 0:
            print('\n', D, K, A, y_approx, la, file = SignChangeFile)
            print('Det Change, lambda = {lamb}'.format(lamb = round(la, 2)))
            print("y_approx = ", y_approx, " la = ", la)  # ------------------------------  print y_approx
            steps, yval, zval = grkutta(system, result[1], np.linspace(-A, A, 100), '')
            #---
            glob_vals.append([steps, yval, zval])
            #----
            print(list(steps), '\n', yval, '\n', zval, '\n', file = SignChangeFile)
            print(la, '\n', file = Lambdas0File)
        y_approx = result[1]
        print("det = ", result[3], file = abcFile)
        lambdas_for_plot.append(la)
        dets.append(result[3])
    abcFile.close()
    SignChangeFile.close()
    Lambdas0File.close()
    return dets, lambdas_for_plot, lambdas_mistake_zerodet, lambdas_mistake_nan, lambdas_mistake_cycle, num_sing_values, lam_steps, first_sinval

In [ ]:
# Runge-Kutta's and Newton-Raphson's methods

def grkutta(f, y_value, steps, title):   # runge_kutta(f, y_value, steps) + graph
    y = np.hstack((steps[0], y_value.copy()))
    yval, zval = [y[1]], [y[2]]
    for i in range(1, len(steps)):
        h = steps[i] - steps[i - 1]
        t1 = steps[i]
        k1 = count(f, y, 0, np.array([0] * (len(y) - 1)))
        k2 = count(f, y, h / 2, k1 * h / 2)
        k3 = count(f, y, h / 2, k2 * h / 2)
        k4 = count(f, y, h, k3 * h)
        y[1:] += h * (k1 + 2 * k2 + 2 * k3 + k4) / 6
        y[0] = t1
        yval.append(y[1])
        zval.append(y[2])

    fig, (yax, zax) = plt.subplots(1, 2)
    yax.scatter(steps, yval, s = 8, color = 'green')
    yax.set_title('Y ' + title)
    zax.scatter(steps, zval, s = 8, color = 'green')
    zax.set_title('Z ' + title)
    plt.show()
    return list(steps), yval, zval

def grkutta_(f, y_value, steps, title):   # runge_kutta(f, y_value, steps) + graph
    y = np.hstack((steps[0], y_value.copy()))
    yval, zval = [y[1]], [y[2]]
    for i in range(1, len(steps)):
        h = steps[i] - steps[i - 1]
        t1 = steps[i]
        k1 = count(f, y, 0, np.array([0] * (len(y) - 1)))
        k2 = count(f, y, h / 2, k1 * h / 2)
        k3 = count(f, y, h / 2, k2 * h / 2)
        k4 = count(f, y, h, k3 * h)
        y[1:] += h * (k1 + 2 * k2 + 2 * k3 + k4) / 6
        y[0] = t1
        yval.append(y[1])
        zval.append(y[2])
    return list(steps), yval, zval



def count(f, args, h, k):
    ''' calculates f(args[0] + h, args[1:] + k) '''
    res = []
    args0 = args.copy()
    args0[0] += h
    args0[1:] += k
    for i in range(len(f)):
        res.append(f[i](args0))
    return np.array(res)


def runge_kutta(f, y_value, steps):
    y = np.hstack((steps[0], y_value.copy()))
    for i in range(1, len(steps)):
        h = steps[i] - steps[i - 1]
        t1 = steps[i]
        k1 = count(f, y, 0, np.array([0] * (len(y) - 1)))
        k2 = count(f, y, h / 2, k1 * h / 2)
        k3 = count(f, y, h / 2, k2 * h / 2)
        k4 = count(f, y, h, k3 * h)
        y[1:] += h * (k1 + 2 * k2 + 2 * k3 + k4) / 6
        y[0] = t1
    return y[1:]



def shooting(time_steps, y_approx, f, params, bc, bc_params, sourceFilename, solver=runge_kutta):
    global la
    boarder_values = bc
    y_value = y_approx
    space = time_steps
    #----
    eps = 1e-4
    step = 0
    
    determ = 0.0
    cnt = 0
    sourceFile = open(sourceFilename, 'a')
    zeroDetFile = open('zeroDet.txt', 'a')
    while True:
        step += 1
        add = runge_kutta(f, y_value, space)
        
        redisuals = np.hstack((y_value, add))     #   невязки
        redisuals = boarder_values(np.hstack((y_value, add)))
        # ----------------------------------------------------------------------------------------------------
        if np.linalg.norm(redisuals) < 1e-6:
           return 0, y_value, add, determ
        # if (np.abs(redisuals) < 1e-6).all():
        #     u,s,v = np.linalg.svd(D, full_matrices=False)
        #     return 0, y_value, add, determ, s
            # break
        # ----------------------------------------------------------------------------------------------------
        D = np.eye(len(f))
        for i in range(len(f)):
            y = y_value.copy()
            y[i] += eps
            dadd = runge_kutta(f, y, space)
            dres = np.hstack((y, dadd))
            # dr = norm(boarder_values, dres)
            dr = boarder_values(dres)
            D[:, i] = (dr - redisuals) / eps
        determ = np.linalg.det(D)

        if abs(determ) < 1e-6:
            print("det ~= 0", D, file = sourceFile)
            print("det ~= 0", D, file = zeroDetFile)
            u,s,v = np.linalg.svd(D, full_matrices=False)
            return 1, y_value, add, determ, s

        if np.isnan(D).any():
            print('nan!!!', file = sourceFile)
            return 2, y_value, add, determ, []
        
        if step > 50:
            print("step > 50", "det =", determ, file = sourceFile)
            return 3, y_value, add, determ, []
        y_value = y_value - np.linalg.pinv(D) @ redisuals       # Newton
    sourceFile.close()

In [ ]:
# border conditions and Euler's equations for the system

def F1(args):
    x, y, z, dy, dz = args[:5]
    return dy
def F2(args):
    global la, k11, k12, k13
    x, y, z, dy, dz = args[:5]
    return (y + k11 * y**2 + 2 * k12 * z * y + k13 * z**2 + la) / D    # d^2y / dx^2
def G1(args):
    x, y, z, dy, dz = args[:5]
    return dz
def G2(args):
    global la, k21, k22, k23
    x, y, z, dy, dz = args[:5]
    return (z + k21 * y**2 + 2 * k22 * z * y + k23 * z**2 + la) / D     # d^2z / dx^2

system = [F1, G1, F2, G2]

def boarder_values(l):
    return np.array([l[0], l[1], l[4], l[5]])

#  0      1      2      3     4    5    6     7   
# y(-A) z(-A) y'(-A) z'(-A) y(A) z(A) y'(A) z'(A)


bc = boarder_values

In [ ]:
# Newton-Raphson's method for parameter changing

def fp_shooting(skip_i, time_steps, y_approx, f, params, bc, bc_params, sourceFilename, solver=runge_kutta):
    # y_approx[skip_i] = const, system[skip_i] - function, which becomes parameter
    global la
    boarder_values = bc
    y_value = y_approx    # space = time_steps
    eps = 1e-3
    step = 0
    determ = 0.0
    zeroDetFile = open('zeroDet.txt', 'a')
    new_lambda = la
    while True:
        step += 1
        left_steps, right_steps = time_steps[len(time_steps)//2::-1], time_steps[len(time_steps) // 2:]
        begin, end = np.array(runge_kutta(f, y_value, left_steps )), np.array(runge_kutta(f, y_value, right_steps ))[1:]
        
        redisuals = boarder_values(np.concatenate((begin, end)))
        if (np.abs(redisuals) < 1e-4).all():  # if np.linalg.norm(redisuals) < 1e-6:
            return 0, y_value, redisuals, determ, new_lambda
        D = np.eye(len(f))
        
        old_la, la = la, la + eps
        y = y_value.copy()
        begin, end = np.array(runge_kutta(f, y, left_steps )), np.array(runge_kutta(f, y, right_steps ))[1:]
        dres = np.hstack((begin, end))
        dr = boarder_values(dres)
        D[:, skip_i] = (dr - redisuals) / eps
        la = old_la
       
        for i in range(len(f)):
            if i == skip_i:
                continue
            y = y_value.copy()
            y[i] += eps
            begin, end = np.array(runge_kutta(f, y, left_steps )), np.array(runge_kutta(f, y, right_steps ))[1:]
            dres = np.hstack((begin, end))
            dr = boarder_values(dres)
            D[:, i] = (dr - redisuals) / eps
        determ = np.linalg.det(D)
        # print(D, determ)
        if abs(determ) < 1e-6:
            print("det ~= 0", D, file = zeroDetFile)
            return 1, y_value, dres, determ, new_lambda
        if np.isnan(D).any():
            return 2, y_value, dres, determ, new_lambda
        if step > 70:
            return 3, y_value, dres, determ, new_lambda
        
        # multiplying on Frechet matrix
        const_saved_y = y_value[skip_i]  # y_value = [y, z, dy, dz],  len = 4
        y_value[skip_i] = la
        y_value = y_value - np.linalg.pinv(D) @ redisuals
        new_lambda = y_value[skip_i]
        y_value[skip_i] = const_saved_y
    sourceFile.close()

In [ ]:
# parameter changing method

def fp_norm_param(fix_i, linsp, D, K, A, init_approx, system, bc, bc_params):
    global k11, k12, k13, k21, k22, k23, la
    k11, k12, k13, k21, k22, k23 = K
    func_values = np.linspace(linsp[0], linsp[1], linsp[2]) 
    lambdas_for_plot, dets = [], []
    y_approx, last_approx = init_approx, init_approx
    last_la = la
    y_norm, z_norm = [], []
    for value in func_values:
        y_approx[fix_i] = value
        result = fp_shooting(fix_i, np.linspace(-A, A, 100), y_approx, system, params, bc, bc_params, filename)
        if result[0] != 0:
            print('mistake at value =', value, 'lambda = ', la)
            continue
        last_approx, last_la = y_approx, la
        steps, yval, zval = grkutta_(system, result[1], np.linspace(-A, A, 100), '')  # из середины !!!!!!!!!!!!!
        y_approx[:fix_i] = result[1][:fix_i]
        y_approx[fix_i + 1 :] = result[1][fix_i + 1 :]
        y_norm.append(np.linalg.norm(yval))
        z_norm.append(np.linalg.norm(zval))
        lambdas_for_plot.append(la)
        dets.append(result[3])
    print('last y_approx', last_approx, 'lambda', last_la)
    return lambdas_for_plot, y_norm, z_norm




def fp_iterate(fix_i, linsp, D, K, A, init_approx, system, bc, draw_nth, filename):
    global k11, k12, k13, k21, k22, k23, la
    k11, k12, k13, k21, k22, k23 = K
    SignChangeFile = open('ProjFiles/SignChange.txt', 'a')
    Lambdas0File = open('ProjFiles/Lambdas0File_{D}_{k11}_{k12}_{k13}.txt'.format(D = D, k11 = k11, k12 = k12, k13 = k13), 'a')
    print('\n', 'linspace ({l1}, {l2}, {l3})------'.format(l1 = linsp[0], l2 = linsp[1], l3 = linsp[2]), '\n', file = Lambdas0File)
    params, bc_params = [], []
    func_values = np.linspace(linsp[0], linsp[1], linsp[2])  # linsp[0] = init_approx[fix_i]
    lambdas_for_plot, dets = [], []
    y_approx = init_approx
    cnt = 0
    old_la = la
    for value in func_values:  # движение по system[fix_i] вместо lambda
        cnt += 1
        y_approx[fix_i] = value
        result = fp_shooting(fix_i, np.linspace(-A, A, 100), y_approx, system, params, bc, bc_params, filename)   # return 0, y_value, add, determ, new_lambda
        if result[0] != 0:
            continue
        if cnt % draw_nth == 0:
            print("system[fix_i] = ", value, "y_approx = ", y_approx, " la = ", la)
            steps, yval, zval = grkutta(system, result[1], np.linspace(-A, A, 100), 'la = {lamb}'.format(lamb = round(la, 2)))
        if len(dets) > 0 and result[3] * dets[-1] < 0:
            print('\n', D, K, A, y_approx, la, '\n', file = SignChangeFile)
            print('Det Change, lambda = {lamb}'.format(lamb = round(la, 2)), '\n', "y_approx = ", y_approx, " la = ", la)
            steps, yval, zval = grkutta(system, result[1], np.linspace(-A, A, 100), '')
            print(list(steps), '\n', yval, '\n', zval, '\n', file = SignChangeFile)
            print(la, '\n', file = Lambdas0File)
        la = result[4]
        y_approx[:fix_i] = result[1][:fix_i]
        y_approx[fix_i + 1 :] = result[1][fix_i + 1 :]
        lambdas_for_plot.append(la)
        dets.append(result[3])
    SignChangeFile.close()
    Lambdas0File.close()
    return dets, lambdas_for_plot

In [ ]:
# lambda-det graph
def scatterNoBlowouts(lambdas_for_plot, dets, Title, plot = False):
    n = len(dets)
    ind = list(filter(lambda i: abs(dets[i]) < 1, list(range(n))))
    
    fig = plt.figure(figsize=(20, 8))
    ax = fig.gca()
    
    x = [lambdas_for_plot[i] for i in ind]
    y = [dets[i] for i in ind]
    d = ((max(y) + 0.1) - (min(y) - 0.1)) / 20
    ax.set_yticks(np.arange(min(y) - 0.1, max(y) + 0.1, d))
    c = ((max(x) + 1) - (min(x) - 1)) / 25
    ax.set_xticks(np.arange(min(x) - 1, max(x) + 1, c))
    #ax.set_xticks(np.arange(-4, 4, 0.25))
    ax.set_xlabel("Lambda")
    ax.set_ylabel("Det")
    plt.scatter(x, y)
    if plot:
        plt.plot(x, y, color = 'gray')
    plt.title(Title)
    plt.grid()
    plt.show()

def scatterAll(lambdas_for_plot, dets, lambdas_mistake_zerodet, lambdas_mistake_nan, lambdas_mistake_cycle, Title, plot = False):
    n = len(dets)
    ind = list(filter(lambda i: abs(dets[i]) < 2, list(range(n))))
    
    fig = plt.figure(figsize=(20, 8))
    ax = fig.gca()
    
    x = [lambdas_for_plot[i] for i in ind]
    y = [dets[i] for i in ind]
    
    ax.set_yticks(np.arange(-1.1, 1.1, 0.1))
    c = ((max(x) + 1) - (min(x) - 1)) / 25
    ax.set_xticks(np.arange(min(x) - 1, max(x) + 1, c))
    #ax.set_xticks(np.arange(-4, 4, 0.25))
    ax.set_xlabel("Lambda")
    ax.set_ylabel("Det")
    plt.scatter(x, y, color = 'steelblue', s = 8)
    plt.scatter(lambdas_mistake_zerodet, [0] * len(lambdas_mistake_zerodet), s = 15, color = 'green', label = 'zero det')
    plt.scatter(lambdas_mistake_nan, [0] * len(lambdas_mistake_nan), s = 8, color = 'orange', label = 'nans in matrix')
    plt.scatter(lambdas_mistake_cycle, [0] * len(lambdas_mistake_cycle), s = 15, color = 'red', label = 'too many steps')
    if plot:
        plt.plot(x, y, color = 'gray')
    plt.title(Title)
    plt.legend()
    plt.grid()
    plt.show()